In [54]:
import plotly.graph_objects as go

from src.Data import Data
from src.Camera import Camera
from src.Plot import Plot
from src.VisualOdometry import VisualOdometry

In [55]:
data = Data()
camera = Camera('data/camera.dat')

In [56]:
vo = VisualOdometry(camera, data)
T, world_points = vo.initialize()
vo.update_state(T, world_points)

# fig = go.Figure()

# fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0], mode='markers', marker=dict(size=3, color='black')))
# fig.add_trace(go.Scatter3d(x=[T[:3,3][0]], y=[T[:3,3][1]], z=[T[:3,3][2]], mode='markers', marker=dict(size=3, color='blue')))
# for pw in world_points:
#     fig.add_trace(go.Scatter3d(x=[pw[0]], y=[pw[1]], z=[pw[2]], mode='markers', marker=dict(size=2, color='orange')))
# fig.show()

In [57]:
import numpy as np
import src.utils as utils

for i in range(120):
    matches = vo.data_association(data.get_measurement_points(i), vo.get_map())
    image_points = matches['points_1']
    world_points = matches['points_2']

    vo.one_step(matches, keep_outliers=False)

trajectory = vo.get_trajectory()
print(len(trajectory['poses']))

121


In [78]:
T0 = np.eye(4)
GT0 = np.eye(4)

vo_odometry = []
gt_odometry = []
ratios = [] 
for i in range(120):
    pose = trajectory['poses'][i]
    T1 = pose
    relT = np.dot(np.linalg.inv(T0), T1)


    meas = data.get_measurements(i)
    gt_pose = meas['gt_pose']
    gt_v = np.array([gt_pose[0], gt_pose[1], gt_pose[2], 0, 0, 0])
    gt_T1 = utils.v2T(gt_v)
    relGT = np.dot(np.linalg.inv(GT0), gt_T1)

    error_T = np.dot(np.linalg.inv(relT), relGT)

    if i==0:
        ratio = 1
        ratios.append(ratio)
    if i>0:
        rot = np.trace(np.eye(3) - error_T[:3,:3])
        ratio = np.linalg.norm(relGT[:3,3]) / np.linalg.norm(relT[:3,3])
        ratios.append(ratio)

    R = T1[:3,:3]
    t = T1[:3,3]
    vo_odometry.append([t[0], t[1], t[2]])
    gt_odometry.append([gt_pose[0], gt_pose[1], gt_pose[2]])

mean_ratio = np.mean(ratios)

In [79]:
plot = Plot()
plot.add_trajectory(gt_odometry, 'Ground Truth', 'green')
plot.add_trajectory(vo_odometry, 'Visual Odometry', 'red')
plot.show()